In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
import torch.nn as nn
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights

# Number of classes: 2 foreground classes + 1 background class
num_classes = 3

# Loading the model with pretrained weights
weights = DeepLabV3_ResNet50_Weights.DEFAULT
model = deeplabv3_resnet50(weights=weights)


model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1))
model.aux_classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1))

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze only the classifier and aux_classifier layers
for param in model.classifier.parameters():
    param.requires_grad = True
for param in model.aux_classifier.parameters():
    param.requires_grad = True


Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:01<00:00, 146MB/s]


In [3]:
import gradio as gr
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(torch.load("deeplabv3_best_model.pth", map_location=device), strict=False)
model = model.to(device)
model.eval()

image_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

color_mapping = {
    0: (0, 0, 0),        # background: black
    1: (255, 0, 0),      # Door: red
    2: (0, 255, 0),      # Stair: green
}

def apply_color_mapping(mask, color_mapping):
    h, w = mask.shape
    colored_mask = np.zeros((h, w, 3), dtype=np.uint8)

    for class_id, color in color_mapping.items():
        colored_mask[mask == class_id] = color

    return colored_mask

def predict_segmentation(image):
    image = image_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)['out']
        prediction = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()

    colored_mask = apply_color_mapping(prediction, color_mapping)

    colored_mask_image = Image.fromarray(colored_mask)

    return colored_mask_image

interface = gr.Interface(
    fn=predict_segmentation,
    inputs=gr.Image(type="pil", label="Input Image"),
    outputs=gr.Image(type="pil", label="Segmented Output"),
    title="DeepLabV3 Semantic Segmentation",
    description="Upload an image to get the semantic segmentated output."
)

# Launch the Gradio app
interface.launch(share=True)


<ipython-input-3-3bad0d5bd046>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("deeplabv3_best_model.pth", map_location=device), strict=Fa

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://379dbe521394e54f75.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
